In [ ]:
eval = [2466, 1257, 5774, 646, 8050, 849, 394, 4174, 203, 139, 4097, 61, 173, 1026, 1422, 1148, 1354, 5082, 343, 184, 366, 1108, 4556, 3438, 363, 604, 1313, 140, 264, 221]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/IR Proj/scraped_CSV.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df['review_sentences'] = df['review_sentences'].map(lambda x : ast.literal_eval(x))


In [ ]:
import math

class UnigramLanguageModel:
    def __init__(self, sentences, mode="collection", smoothing=False):

        '''
            sentences: sentences of the dataset
            mode: whether this language model is for the whole corpus/collection or just a single document
            smoothing: add-one smoothing
        '''

        self.sentences = sentences
        self.N = 0
        self.tf = {}

        # for sentence in self.sentences:
        for word in sentences:
          # print(word)
          if word in self.tf:
            self.tf[word]+=1
          else:
            self.tf[word] = 1
          self.N += 1


    def calculate_unigram_probability(self, word):
        '''
            calculate unigram probability of a word
        '''
        return self.tf.get(word,0)/ self.N        
        

    def calculate_sentence_probability(self, sentence, normalize_probability=True):
        '''
            calculate score/probability of a sentence or query using the unigram language model.
            sentence: input sentence or query
            normalize_probability: If true then log of probability is not computed. Otherwise take log2 of the               probability score.
        '''
        word_probabilities = []

        for i in sentence:
          word_probabilities.append(self.calculate_unigram_probability(i))

        if normalize_probability == False:
          product = 0
          for i in word_probabilities:
            product += math.log2(i) if i!= 0 else 0

        else:
          product = 1
          for i in word_probabilities:
            produce = product*i

        return product 

In [ ]:
def calculate_interpolated_sentence_probability(sentence, doc, collection, alpha=0.75, normalize_probability=True):
    '''
        calculate interpolated sentence/query probability using both sentence and collection unigram models.
        sentence: input sentence/query
        doc: unigram language model a doc. HINT: this can be an instance of the UnigramLanguageModel class
        collection: unigram language model a collection. HINT: this can be an instance of the   
        UnigramLanguageModel class
        alpha: the hyperparameter to combine the two probability scores coming from the document and collection 
        language models.
        normalize_probability: If true then log of probability is not computed. Otherwise take log2 of the 
        probability score.
    '''
    scores = []

    for i in sentence:
      scores.append((alpha*doc.calculate_unigram_probability(i) + (1-alpha *collection.calculate_unigram_probability(i))))
      result = 1
      
      if normalize_probability == True:
        result = 1
      for i in scores:
        result *= i
        
    else:
      result = 0
      for i in scores:
        result += math.log2(i) if i!= 0 else 0

    return result

In [ ]:
import ast
UnigramLanguageModel(ast.literal_eval(df.review_sentences[7422])).tf
# a.tf
# import json
# json.loads(df.review_sentences[7422])

# a = ast.literal_eval()
# a = [n.strip() for n in df.review_sentences[7422]]
# print(a)
# ast.literal_eval(df.review_sentences[7422])

In [ ]:
collection_model = UnigramLanguageModel([item for sublist in df['review_sentences'].tolist() for item in sublist], mode="collection", smoothing=False)

In [ ]:
ulm_dict = {}

def calc_ulm(row):
    ulm_dict[row.title] = UnigramLanguageModel(row.review_sentences)


In [ ]:
df.apply(calc_ulm, axis=1)

0        None
1        None
2        None
3        None
4        None
         ... 
13564    None
13565    None
13566    None
13567    None
13568    None
Length: 13569, dtype: object

In [ ]:
a = UnigramLanguageModel(df.review_sentences[7422])
a.tf

In [ ]:
# ranks = []
# def calc_rankings:
#   for i in eval:
#     book = df.loc[[i]]
#     for other in df.iterrows():

    
rankings = {}

for i in eval[0:10]:
      print(i)
      # book = df.loc[[i]] #Get row based on index
      rank = []

      for j,other in df.iterrows():
          print(j)
          if i == j:
              continue
          s = calculate_interpolated_sentence_probability(df.review_sentences[i], ulm_dict[other.title], collection_model, alpha=0.75, normalize_probability=False)
          rank.append((other['title'], s))
        
      rank = sorted(rank, key=lambda x: x[1], reverse = True)
      rankings[df.title[i]] = rank

In [ ]:
!pip install futures

In [ ]:
import futures
import concurrent
rankings = {}

def mfunc(i):
  print(i)
  # book = df.loc[[i]] #Get row based on index
  rank = []

  for j,other in df.iterrows():
      if i == j:
          continue
      s = calculate_interpolated_sentence_probability(df.review_sentences[i], ulm_dict[other.title], collection_model, alpha=0.75, normalize_probability=False)
      rank.append((other['title'], s))
    
  rank = sorted(rank, key=lambda x: x[1], reverse = True)[0:15]
  rankings[df.title[i]] = rank

executor = concurrent.futures.ProcessPoolExecutor(10)
futures = [executor.submit(mfunc, item) for item in eval[0:1]]
concurrent.futures.wait(futures)


In [ ]:
rankings

{'Harry Potter and the Order of the Phoenix': [('Bonjour tristesse', 0.0),
  ('The Tommyknockers', 0.0),
  ('Stoner', 0.0),
  ('Emma', 0.0),
  ('Foundation', 0.0),
  ('To Have and Have Not', 0.0),
  ('Half Bad', 0.0),
  ('A Murder is Announced', 0.0),
  ('On the Island', 0.0),
  ('The Crimson Petal and the White', 0.0),
  ('Twelfth Grade Kills', 0.0),
  ('Labyrinth', 0.0),
  ('4.50 From Paddington', 0.0),
  ('White Fire', 0.0),
  ('Robots and Empire', 0.0),
  ('A Monster Calls', 0.0),
  ('The Possessed (Dark Visions, #2)', 0.0),
  ('Prestuplenie i nakazanie', 0.0),
  ('The Saturdays', 0.0),
  ('Tarzan of the Apes', 0.0),
  ('Triptych', 0.0),
  ('Worth the Wait', 0.0),
  ('Books of Blood: Volume One', 0.0),
  ('Left To Die', 0.0),
  ('The White Giraffe', 0.0),
  ('Passion', 0.0),
  ('In The Heat of The Bite', 0.0),
  ('Secrets of a Summer Night', 0.0),
  ('Personal Demon', 0.0),
  ('Slow Hands', 0.0),
  ('Solaris', 0.0),
  ('The Grownup', 0.0),
  ("Fortune's Pawn", 0.0),
  ('Bayou Moon'